# Unit test

In [2]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

/home/ubuntu/varios/skforecast


In [3]:
import numpy as np
import pandas as pd
from copy import deepcopy
from sklearn.metrics import mean_absolute_percentage_error
from skforecast.metrics import mean_absolute_scaled_error
from skforecast.metrics import root_mean_squared_scaled_error
from sklearn.model_selection import ParameterGrid
from sklearn.linear_model import Ridge
from lightgbm import LGBMRegressor
from sklearn.preprocessing import StandardScaler
from skforecast.ForecasterAutoregMultiSeries import ForecasterAutoregMultiSeries
from skforecast.ForecasterAutoregMultiVariate import ForecasterAutoregMultiVariate
from skforecast.model_selection_multiseries.model_selection_multiseries import _evaluate_grid_hyperparameters_multiseries
from skforecast.model_selection_multiseries.model_selection_multiseries import _predict_and_calculate_metrics_multiseries_one_step_ahead

# Fixtures
from skforecast.model_selection.tests.fixtures_model_selection import y_feature_selection
from skforecast.model_selection.tests.fixtures_model_selection import exog_feature_selection

from skforecast.datasets import fetch_dataset



In [89]:
def test_train_test_split_one_step_ahead_when_y_is_series_and_exog_are_dataframe_encoding_none():
    """
    Test the output of _train_test_split_one_step_ahead when series and exog are
    pandas dataframes, and encoding is None.
    """
    series = pd.DataFrame(
        {
            "series_1": np.arange(15),
            "series_2": np.arange(50, 65),
        },
        index=pd.date_range("2020-01-01", periods=15),
        dtype=float,
    )
    exog = pd.DataFrame(
        {
            "exog_1": np.arange(100, 115, dtype=float),
            "exog_2": np.arange(1000, 1015, dtype=int),
        },
        index=pd.date_range("2020-01-01", periods=15),
    )

    forecaster = ForecasterAutoregMultiSeries(
        LinearRegression(),
        lags=5,
        encoding=None,
    )

    X_train, y_train, X_test, y_test, X_train_encoding, X_test_encoding = (
        forecaster._train_test_split_one_step_ahead(
            series=series, exog=exog, initial_train_size=10
        )
    )

    expected_X_trian = pd.DataFrame(
        {
            "lag_1": [4.0, 5.0, 6.0, 7.0, 8.0, 54.0, 55.0, 56.0, 57.0, 58.0],
            "lag_2": [3.0, 4.0, 5.0, 6.0, 7.0, 53.0, 54.0, 55.0, 56.0, 57.0],
            "lag_3": [2.0, 3.0, 4.0, 5.0, 6.0, 52.0, 53.0, 54.0, 55.0, 56.0],
            "lag_4": [1.0, 2.0, 3.0, 4.0, 5.0, 51.0, 52.0, 53.0, 54.0, 55.0],
            "lag_5": [0.0, 1.0, 2.0, 3.0, 4.0, 50.0, 51.0, 52.0, 53.0, 54.0],
            "exog_1": [
                105.0,
                106.0,
                107.0,
                108.0,
                109.0,
                105.0,
                106.0,
                107.0,
                108.0,
                109.0,
            ],
            "exog_2": [1005, 1006, 1007, 1008, 1009, 1005, 1006, 1007, 1008, 1009],
        },
        index=pd.DatetimeIndex(
            [
                "2020-01-06",
                "2020-01-07",
                "2020-01-08",
                "2020-01-09",
                "2020-01-10",
                "2020-01-06",
                "2020-01-07",
                "2020-01-08",
                "2020-01-09",
                "2020-01-10",
            ]
        ),
    ).astype({"exog_2": int})

    expected_y_train = pd.Series(
        [5.0, 6.0, 7.0, 8.0, 9.0, 55.0, 56.0, 57.0, 58.0, 59.0],
        index=pd.DatetimeIndex(
            [
                "2020-01-06",
                "2020-01-07",
                "2020-01-08",
                "2020-01-09",
                "2020-01-10",
                "2020-01-06",
                "2020-01-07",
                "2020-01-08",
                "2020-01-09",
                "2020-01-10",
            ]
        ),
        name="y",
    )

    expected_X_test = pd.DataFrame(
        {
            "lag_1": [9.0, 10.0, 11.0, 12.0, 13.0, 59.0, 60.0, 61.0, 62.0, 63.0],
            "lag_2": [8.0, 9.0, 10.0, 11.0, 12.0, 58.0, 59.0, 60.0, 61.0, 62.0],
            "lag_3": [7.0, 8.0, 9.0, 10.0, 11.0, 57.0, 58.0, 59.0, 60.0, 61.0],
            "lag_4": [6.0, 7.0, 8.0, 9.0, 10.0, 56.0, 57.0, 58.0, 59.0, 60.0],
            "lag_5": [5.0, 6.0, 7.0, 8.0, 9.0, 55.0, 56.0, 57.0, 58.0, 59.0],
            "exog_1": [
                110.0,
                111.0,
                112.0,
                113.0,
                114.0,
                110.0,
                111.0,
                112.0,
                113.0,
                114.0,
            ],
            "exog_2": [1010, 1011, 1012, 1013, 1014, 1010, 1011, 1012, 1013, 1014],
        },
        index=pd.DatetimeIndex(
            [
                "2020-01-11",
                "2020-01-12",
                "2020-01-13",
                "2020-01-14",
                "2020-01-15",
                "2020-01-11",
                "2020-01-12",
                "2020-01-13",
                "2020-01-14",
                "2020-01-15",
            ]
        ),
    ).astype({"exog_2": int})

    expected_y_test = pd.Series(
        [10.0, 11.0, 12.0, 13.0, 14.0, 60.0, 61.0, 62.0, 63.0, 64.0],
        index=pd.DatetimeIndex(
            [
                "2020-01-11",
                "2020-01-12",
                "2020-01-13",
                "2020-01-14",
                "2020-01-15",
                "2020-01-11",
                "2020-01-12",
                "2020-01-13",
                "2020-01-14",
                "2020-01-15",
            ]
        ),
        name="y",
    )

    expected_X_train_encoding = pd.Series(
        [
            "series_1",
            "series_1",
            "series_1",
            "series_1",
            "series_1",
            "series_2",
            "series_2",
            "series_2",
            "series_2",
            "series_2",
        ],
        index=pd.DatetimeIndex(
            [
                "2020-01-06",
                "2020-01-07",
                "2020-01-08",
                "2020-01-09",
                "2020-01-10",
                "2020-01-06",
                "2020-01-07",
                "2020-01-08",
                "2020-01-09",
                "2020-01-10",
            ]
        ),
    )

    expected_X_test_encoding = pd.Series(
        [
            "series_1",
            "series_1",
            "series_1",
            "series_1",
            "series_1",
            "series_2",
            "series_2",
            "series_2",
            "series_2",
            "series_2",
        ],
        index=pd.DatetimeIndex(
            [
                "2020-01-11",
                "2020-01-12",
                "2020-01-13",
                "2020-01-14",
                "2020-01-15",
                "2020-01-11",
                "2020-01-12",
                "2020-01-13",
                "2020-01-14",
                "2020-01-15",
            ]
        ),
    )

    pd.testing.assert_frame_equal(X_train, expected_X_trian)
    pd.testing.assert_series_equal(y_train, expected_y_train)
    pd.testing.assert_frame_equal(X_test, expected_X_test)
    pd.testing.assert_series_equal(y_test, expected_y_test)
    pd.testing.assert_series_equal(X_train_encoding, expected_X_train_encoding)
    pd.testing.assert_series_equal(X_test_encoding, expected_X_test_encoding)


test_train_test_split_one_step_ahead_when_y_is_series_and_exog_are_dataframe_encoding_none()

/home/ubuntu/varios/skforecast/skforecast/ForecasterAutoregMultiSeries/ForecasterAutoregMultiSeries.py:400: UserWarning: When using a linear model, it is recommended to use a transformer_series to ensure all series are in the same scale. You can use, for example, a `StandardScaler` from sklearn.preprocessing.
  warnings.warn(


In [88]:
series = pd.DataFrame(
    {
        "series_1": np.arange(15),
        "series_2": np.arange(50, 65),
    },
    index=pd.date_range("2020-01-01", periods=15),
    dtype=float,
)
exog = pd.DataFrame(
    {
        "exog_1": np.arange(100, 115, dtype=float),
        "exog_2": np.arange(1000, 1015, dtype=int),
    },
    index=pd.date_range("2020-01-01", periods=15),
)

forecaster = ForecasterAutoregMultiSeries(
    LinearRegression(),
    lags=5,
    encoding=None,
)

X_train, y_train, X_test, y_test, X_train_encoding, X_test_encoding = (
    forecaster._train_test_split_one_step_ahead(
        series=series, exog=exog, initial_train_size=10
    )
)

expected_X_trian = pd.DataFrame(
    {
        "lag_1": [4.0, 5.0, 6.0, 7.0, 8.0, 54.0, 55.0, 56.0, 57.0, 58.0],
        "lag_2": [3.0, 4.0, 5.0, 6.0, 7.0, 53.0, 54.0, 55.0, 56.0, 57.0],
        "lag_3": [2.0, 3.0, 4.0, 5.0, 6.0, 52.0, 53.0, 54.0, 55.0, 56.0],
        "lag_4": [1.0, 2.0, 3.0, 4.0, 5.0, 51.0, 52.0, 53.0, 54.0, 55.0],
        "lag_5": [0.0, 1.0, 2.0, 3.0, 4.0, 50.0, 51.0, 52.0, 53.0, 54.0],
        "_level_skforecast": [0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
        "exog_1": [
            105.0,
            106.0,
            107.0,
            108.0,
            109.0,
            105.0,
            106.0,
            107.0,
            108.0,
            109.0,
        ],
        "exog_2": [1005, 1006, 1007, 1008, 1009, 1005, 1006, 1007, 1008, 1009],
    },
    index=pd.DatetimeIndex(
        [
            "2020-01-06",
            "2020-01-07",
            "2020-01-08",
            "2020-01-09",
            "2020-01-10",
            "2020-01-06",
            "2020-01-07",
            "2020-01-08",
            "2020-01-09",
            "2020-01-10",
        ]
    ),
).astype({"exog_2": int})

expected_y_train = pd.Series(
    [5.0, 6.0, 7.0, 8.0, 9.0, 55.0, 56.0, 57.0, 58.0, 59.0],
    index=pd.DatetimeIndex(
        [
            "2020-01-06",
            "2020-01-07",
            "2020-01-08",
            "2020-01-09",
            "2020-01-10",
            "2020-01-06",
            "2020-01-07",
            "2020-01-08",
            "2020-01-09",
            "2020-01-10",
        ]
    ),
    name="y",
)

expected_X_test = pd.DataFrame(
    {
        "lag_1": [9.0, 10.0, 11.0, 12.0, 13.0, 59.0, 60.0, 61.0, 62.0, 63.0],
        "lag_2": [8.0, 9.0, 10.0, 11.0, 12.0, 58.0, 59.0, 60.0, 61.0, 62.0],
        "lag_3": [7.0, 8.0, 9.0, 10.0, 11.0, 57.0, 58.0, 59.0, 60.0, 61.0],
        "lag_4": [6.0, 7.0, 8.0, 9.0, 10.0, 56.0, 57.0, 58.0, 59.0, 60.0],
        "lag_5": [5.0, 6.0, 7.0, 8.0, 9.0, 55.0, 56.0, 57.0, 58.0, 59.0],
        "_level_skforecast": [0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
        "exog_1": [
            110.0,
            111.0,
            112.0,
            113.0,
            114.0,
            110.0,
            111.0,
            112.0,
            113.0,
            114.0,
        ],
        "exog_2": [1010, 1011, 1012, 1013, 1014, 1010, 1011, 1012, 1013, 1014],
    },
    index=pd.DatetimeIndex(
        [
            "2020-01-11",
            "2020-01-12",
            "2020-01-13",
            "2020-01-14",
            "2020-01-15",
            "2020-01-11",
            "2020-01-12",
            "2020-01-13",
            "2020-01-14",
            "2020-01-15",
        ]
    ),
).astype({"exog_2": int})

expected_y_test = pd.Series(
    [10.0, 11.0, 12.0, 13.0, 14.0, 60.0, 61.0, 62.0, 63.0, 64.0],
    index=pd.DatetimeIndex(
        [
            "2020-01-11",
            "2020-01-12",
            "2020-01-13",
            "2020-01-14",
            "2020-01-15",
            "2020-01-11",
            "2020-01-12",
            "2020-01-13",
            "2020-01-14",
            "2020-01-15",
        ]
    ),
    name="y",
)

expected_X_train_encoding = pd.Series(
    [
        "series_1",
        "series_1",
        "series_1",
        "series_1",
        "series_1",
        "series_2",
        "series_2",
        "series_2",
        "series_2",
        "series_2",
    ],
    index=pd.DatetimeIndex(
        [
            "2020-01-06",
            "2020-01-07",
            "2020-01-08",
            "2020-01-09",
            "2020-01-10",
            "2020-01-06",
            "2020-01-07",
            "2020-01-08",
            "2020-01-09",
            "2020-01-10",
        ]
    ),
)

expected_X_test_encoding = pd.Series(
    [
        "series_1",
        "series_1",
        "series_1",
        "series_1",
        "series_1",
        "series_2",
        "series_2",
        "series_2",
        "series_2",
        "series_2",
    ],
    index=pd.DatetimeIndex(
        [
            "2020-01-11",
            "2020-01-12",
            "2020-01-13",
            "2020-01-14",
            "2020-01-15",
            "2020-01-11",
            "2020-01-12",
            "2020-01-13",
            "2020-01-14",
            "2020-01-15",
        ]
    ),
)

X_test.to_dict(orient='list')

/home/ubuntu/varios/skforecast/skforecast/ForecasterAutoregMultiSeries/ForecasterAutoregMultiSeries.py:400: UserWarning: When using a linear model, it is recommended to use a transformer_series to ensure all series are in the same scale. You can use, for example, a `StandardScaler` from sklearn.preprocessing.
  warnings.warn(


{'lag_1': [9.0, 10.0, 11.0, 12.0, 13.0, 59.0, 60.0, 61.0, 62.0, 63.0],
 'lag_2': [8.0, 9.0, 10.0, 11.0, 12.0, 58.0, 59.0, 60.0, 61.0, 62.0],
 'lag_3': [7.0, 8.0, 9.0, 10.0, 11.0, 57.0, 58.0, 59.0, 60.0, 61.0],
 'lag_4': [6.0, 7.0, 8.0, 9.0, 10.0, 56.0, 57.0, 58.0, 59.0, 60.0],
 'lag_5': [5.0, 6.0, 7.0, 8.0, 9.0, 55.0, 56.0, 57.0, 58.0, 59.0],
 'exog_1': [110.0,
  111.0,
  112.0,
  113.0,
  114.0,
  110.0,
  111.0,
  112.0,
  113.0,
  114.0],
 'exog_2': [1010, 1011, 1012, 1013, 1014, 1010, 1011, 1012, 1013, 1014]}

In [76]:
# Repeat this code but the series should not have frequency
# series = {
#             "series_1": pd.Series(np.arange(15, dtype=float), index=pd.date_range("2020-01-01", periods=15)),
#             "series_2": pd.Series(np.arange(50, 65, dtype=float), index=pd.date_range("2020-01-01", periods=15))
#         }

series = {
            "series_1": pd.Series(np.arange(15, dtype=float), index=pd.date_range("2020-01-01", periods=15, freq="D")),
            "series_2": pd.Series(np.arange(50, 65, dtype=float), index=pd.date_range("2020-01-01", periods=15, freq="H"))
        }

series['series_1'].index.freq = None
series['series_2'].index.freq = None
series


/tmp/ipykernel_5421/2840598471.py:9: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  "series_2": pd.Series(np.arange(50, 65, dtype=float), index=pd.date_range("2020-01-01", periods=15, freq="H"))


{'series_1': 2020-01-01     0.0
 2020-01-02     1.0
 2020-01-03     2.0
 2020-01-04     3.0
 2020-01-05     4.0
 2020-01-06     5.0
 2020-01-07     6.0
 2020-01-08     7.0
 2020-01-09     8.0
 2020-01-10     9.0
 2020-01-11    10.0
 2020-01-12    11.0
 2020-01-13    12.0
 2020-01-14    13.0
 2020-01-15    14.0
 dtype: float64,
 'series_2': 2020-01-01 00:00:00    50.0
 2020-01-01 01:00:00    51.0
 2020-01-01 02:00:00    52.0
 2020-01-01 03:00:00    53.0
 2020-01-01 04:00:00    54.0
 2020-01-01 05:00:00    55.0
 2020-01-01 06:00:00    56.0
 2020-01-01 07:00:00    57.0
 2020-01-01 08:00:00    58.0
 2020-01-01 09:00:00    59.0
 2020-01-01 10:00:00    60.0
 2020-01-01 11:00:00    61.0
 2020-01-01 12:00:00    62.0
 2020-01-01 13:00:00    63.0
 2020-01-01 14:00:00    64.0
 Freq: h, dtype: float64}

In [52]:
expected_X_trian = pd.DataFrame(
        {
            'lag_1': [4.0, 5.0, 6.0, 7.0, 8.0, 54.0, 55.0, 56.0, 57.0, 58.0],
            'lag_2': [3.0, 4.0, 5.0, 6.0, 7.0, 53.0, 54.0, 55.0, 56.0, 57.0],
            'lag_3': [2.0, 3.0, 4.0, 5.0, 6.0, 52.0, 53.0, 54.0, 55.0, 56.0],
            'lag_4': [1.0, 2.0, 3.0, 4.0, 5.0, 51.0, 52.0, 53.0, 54.0, 55.0],
            'lag_5': [0.0, 1.0, 2.0, 3.0, 4.0, 50.0, 51.0, 52.0, 53.0, 54.0],
            '_level_skforecast': [0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
            'exog_1': [105.0,
            106.0,
            107.0,
            108.0,
            109.0,
            105.0,
            106.0,
            107.0,
            108.0,
            109.0],
            'exog_2': [1005, 1006, 1007, 1008, 1009, 1005, 1006, 1007, 1008, 1009]
        },
        index=pd.DatetimeIndex(
            ['2020-01-06', '2020-01-07', '2020-01-08', '2020-01-09', '2020-01-10',
             '2020-01-06', '2020-01-07', '2020-01-08', '2020-01-09', '2020-01-10']
        ),
    ).astype({"exog_2": int})

expected_X_trian.equals(X_train)

True

In [4]:
metrics = ['mean_absolute_error', mean_absolute_percentage_error, mean_absolute_scaled_error]
data = fetch_dataset(name="items_sales", verbose=False)
data['day_of_week'] = data.index.dayofweek
end_train = "2014-07-15 23:59:00"
initial_train_size = len(data.loc[:end_train, :].copy())
data_test = data.loc[end_train:, :].copy()
levels = ["item_1", "item_2", "item_3"]
exog_features = 'day_of_week'

metrics = [
    "mean_absolute_error",
    "mean_squared_error",
    mean_absolute_percentage_error,
    mean_absolute_scaled_error,
    root_mean_squared_scaled_error,
]
steps = 1
initial_train_size = 100
param_grid = {
    "alpha": np.logspace(-3, 3, 1),
}
lags_grid = [5]
param_grid = list(ParameterGrid(param_grid))

forecaster = ForecasterAutoregMultiSeries(
    regressor=Ridge(random_state=678),
    lags=5,
    transformer_series=StandardScaler(),
    transformer_exog=StandardScaler(),
)

forecaster = ForecasterAutoregMultiVariate(
    regressor=Ridge(random_state=678),
    lags=3,
    steps=1,
    level="item_1",
    transformer_series=StandardScaler(),
    transformer_exog=StandardScaler(),
)

X_train, y_train, X_test, y_test, X_train_encoding, X_test_encoding = (
    forecaster._train_test_split_one_step_ahead(
        series=data.loc[:, levels],
        exog=data.loc[:, exog_features] if exog_features else None,
        initial_train_size=initial_train_size,
    )
)

results_one_step_ahead = _evaluate_grid_hyperparameters_multiseries(
        forecaster         = forecaster,
        series             = data.loc[:, levels],
        exog               = data.loc[:, exog_features] if exog_features else None,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'one_step_ahead',
        return_best        = False,
        verbose            = False,
        show_progress      = True
    )

results_one_step_ahead

/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1802: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


lags grid:   0%|          | 0/1 [00:00<?, ?it/s]

params grid:   0%|          | 0/1 [00:00<?, ?it/s]

,levels,lags,lags_label,params,mean_absolute_error,mean_squared_error,mean_absolute_percentage_error,mean_absolute_scaled_error,root_mean_squared_scaled_error,alpha
0,[item_1],"[1, 2, 3, 4, 5]","[1, 2, 3, 4, 5]",{'alpha': 0.001},1.154476,2.582983,0.051834,0.774511,0.817991,0.001


In [5]:
data = fetch_dataset(name="items_sales", verbose=False)
data['day_of_week'] = data.index.dayofweek
end_train = "2014-07-15 23:59:00"
initial_train_size = len(data.loc[:end_train, :].copy())
levels = ["item_1", "item_2", "item_3"]
exog_features = ['day_of_week']

def test_evaluate_grid_hyperparameters_equivalent_outputs_backtesting_one_step_ahead(
    forecaster,
):

    metrics = [
        "mean_absolute_error",
        "mean_squared_error",
        mean_absolute_percentage_error,
        mean_absolute_scaled_error,
        root_mean_squared_scaled_error,
    ]
    steps = 1
    initial_train_size = 1000
    param_grid = {
        "alpha": np.logspace(-1, 1, 3),
    }
    lags_grid = [3, 7]
    param_grid = list(ParameterGrid(param_grid))
    results_backtesting = _evaluate_grid_hyperparameters_multiseries(
        forecaster         = deepcopy(forecaster),
        series             = data.loc[:, levels],
        exog               = data.loc[:, exog_features] if exog_features else None,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        steps              = steps,
        refit              = False,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'backtesting',
        aggregate_metric   = ["average", "weighted_average", "pooling"],
        fixed_train_size   = False,
        return_best        = False,
        n_jobs             = 'auto',
        verbose            = False,
        show_progress      = False
    )
    display(results_backtesting)
    results_one_step_ahead = _evaluate_grid_hyperparameters_multiseries(
        forecaster         = deepcopy(forecaster),
        series             = data.loc[:, levels],
        exog               = data.loc[:, exog_features] if exog_features else None,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'one_step_ahead',
        aggregate_metric   = ["average", "weighted_average", "pooling"],
        return_best        = False,
        verbose            = False,
        show_progress      = False
    )

    display(results_backtesting)
    pd.testing.assert_frame_equal(results_backtesting, results_one_step_ahead)


regressor = Ridge(random_state=678)
forecasters = [
    # Diferenciation must be 0 for this test
    ForecasterAutoregMultiSeries(regressor=regressor, lags=3),
    ForecasterAutoregMultiSeries(
        regressor=regressor,
        lags=3,
        transformer_series=None,
    ),
    ForecasterAutoregMultiSeries(
        regressor=regressor,
        lags=3,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler()
    ),
    ForecasterAutoregMultiVariate(
        regressor=regressor,
        level='item_1',
        lags=3,
        steps=1,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler()
    )

]

for i, forecaster in enumerate(forecasters):
    print(i)
    test_evaluate_grid_hyperparameters_equivalent_outputs_backtesting_one_step_ahead(
        forecaster=forecaster
    )

0


/home/ubuntu/varios/skforecast/skforecast/ForecasterAutoregMultiSeries/ForecasterAutoregMultiSeries.py:400: UserWarning: When using a linear model, it is recommended to use a transformer_series to ensure all series are in the same scale. You can use, for example, a `StandardScaler` from sklearn.preprocessing.
  warnings.warn(


KeyboardInterrupt: 

In [7]:
from skforecast.model_selection_multiseries.tests.fixtures_model_selection_multiseries import series
from skforecast.model_selection_multiseries.tests.fixtures_model_selection_multiseries import exog

series.index = pd.date_range(start='2024-01-01', periods=len(series), freq='D')
exog.index = pd.date_range(start='2024-01-01', periods=len(exog), freq='D')

def test_evaluate_grid_hyperparameters_equivalent_outputs_backtesting_one_step_ahead(
    forecaster,
):

    metrics = [
        "mean_absolute_error",
        "mean_squared_error",
        mean_absolute_percentage_error,
        mean_absolute_scaled_error,
        root_mean_squared_scaled_error,
    ]
    steps = 1
    initial_train_size = 20
    param_grid = {
        "alpha": np.logspace(-1, 1, 3),
    }
    lags_grid = [3, 7]
    param_grid = list(ParameterGrid(param_grid))
    results_backtesting = _evaluate_grid_hyperparameters_multiseries(
        forecaster         = forecaster,
        series             = series,
        exog               = exog,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        steps              = steps,
        refit              = False,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'backtesting',
        aggregate_metric   = ["average", "weighted_average", "pooling"],
        fixed_train_size   = False,
        return_best        = False,
        n_jobs             = 'auto',
        verbose            = False,
        show_progress      = False
    )
    display(results_backtesting)
    results_one_step_ahead = _evaluate_grid_hyperparameters_multiseries(
        forecaster         = forecaster,
        series             = series,
        exog               = exog,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'one_step_ahead',
        aggregate_metric   = ["average", "weighted_average", "pooling"],
        return_best        = False,
        verbose            = False,
        show_progress      = False
    )
    display(results_backtesting)

    pd.testing.assert_frame_equal(results_backtesting, results_one_step_ahead)


regressor = Ridge(random_state=678)
forecasters = [
    # Diferenciation must be 0 for this test
    ForecasterAutoregMultiSeries(regressor=regressor, lags=3),
    ForecasterAutoregMultiSeries(
        regressor=regressor,
        lags=3,
        transformer_series=None,
    ),
    ForecasterAutoregMultiSeries(
        regressor=regressor,
        lags=3,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler()
    ),
    ForecasterAutoregMultiVariate(
        regressor=regressor,
        level='l1',
        lags=3,
        steps=1,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler()
    )
]

for i, forecaster in enumerate(forecasters):
    print(i)
    test_evaluate_grid_hyperparameters_equivalent_outputs_backtesting_one_step_ahead(
        forecaster=forecaster
    )

0


/home/ubuntu/varios/skforecast/skforecast/ForecasterAutoregMultiSeries/ForecasterAutoregMultiSeries.py:400: UserWarning: When using a linear model, it is recommended to use a transformer_series to ensure all series are in the same scale. You can use, for example, a `StandardScaler` from sklearn.preprocessing.
  warnings.warn(


,levels,lags,lags_label,params,mean_absolute_error__average,mean_absolute_error__weighted_average,mean_absolute_error__pooling,mean_squared_error__average,mean_squared_error__weighted_average,mean_squared_error__pooling,mean_absolute_percentage_error__average,mean_absolute_percentage_error__weighted_average,mean_absolute_percentage_error__pooling,mean_absolute_scaled_error__average,mean_absolute_scaled_error__weighted_average,mean_absolute_scaled_error__pooling,root_mean_squared_scaled_error__average,root_mean_squared_scaled_error__weighted_average,root_mean_squared_scaled_error__pooling,alpha
0,"[l1, l2]","[1, 2, 3]","[1, 2, 3]",{'alpha': 0.1},0.181897,0.181897,0.181897,0.048689,0.048689,0.048689,1.034332,1.034332,1.034332,0.696273,0.696273,0.694963,0.673709,0.673709,0.674524,0.1
1,"[l1, l2]","[1, 2, 3]","[1, 2, 3]",{'alpha': 1.0},0.187109,0.187109,0.187109,0.050646,0.050646,0.050646,1.114850,1.114850,1.114850,0.716711,0.716711,0.714877,0.686177,0.686177,0.687946,1.0
2,"[l1, l2]","[1, 2, 3, 4, 5, 6, 7]","[1, 2, 3, 4, 5, 6, 7]",{'alpha': 0.1},0.199475,0.199475,0.199475,0.061923,0.061923,0.061923,1.171530,1.171530,1.171530,0.745361,0.745361,0.743463,0.738244,0.738244,0.730532,0.1
3,"[l1, l2]","[1, 2, 3, 4, 5, 6, 7]","[1, 2, 3, 4, 5, 6, 7]",{'alpha': 1.0},0.200192,0.200192,0.200192,0.058144,0.058144,0.058144,1.239896,1.239896,1.239896,0.746616,0.746616,0.746137,0.715926,0.715926,0.707888,1.0
4,"[l1, l2]","[1, 2, 3]","[1, 2, 3]",{'alpha': 10.0},0.206143,0.206143,0.206143,0.060303,0.060303,0.060303,1.267311,1.267311,1.267311,0.790667,0.790667,0.787598,0.747907,0.747907,0.750675,10.0
5,"[l1, l2]","[1, 2, 3, 4, 5, 6, 7]","[1, 2, 3, 4, 5, 6, 7]",{'alpha': 10.0},0.207645,0.207645,0.207645,0.062203,0.062203,0.062203,1.294716,1.294716,1.294716,0.773001,0.773001,0.773914,0.738505,0.738505,0.732181,10.0


/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1802: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


,levels,lags,lags_label,params,mean_absolute_error__average,mean_absolute_error__weighted_average,mean_absolute_error__pooling,mean_squared_error__average,mean_squared_error__weighted_average,mean_squared_error__pooling,mean_absolute_percentage_error__average,mean_absolute_percentage_error__weighted_average,mean_absolute_percentage_error__pooling,mean_absolute_scaled_error__average,mean_absolute_scaled_error__weighted_average,mean_absolute_scaled_error__pooling,root_mean_squared_scaled_error__average,root_mean_squared_scaled_error__weighted_average,root_mean_squared_scaled_error__pooling,alpha
0,"[l1, l2]","[1, 2, 3]","[1, 2, 3]",{'alpha': 0.1},0.181897,0.181897,0.181897,0.048689,0.048689,0.048689,1.034332,1.034332,1.034332,0.696273,0.696273,0.694963,0.673709,0.673709,0.674524,0.1
1,"[l1, l2]","[1, 2, 3]","[1, 2, 3]",{'alpha': 1.0},0.187109,0.187109,0.187109,0.050646,0.050646,0.050646,1.114850,1.114850,1.114850,0.716711,0.716711,0.714877,0.686177,0.686177,0.687946,1.0
2,"[l1, l2]","[1, 2, 3, 4, 5, 6, 7]","[1, 2, 3, 4, 5, 6, 7]",{'alpha': 0.1},0.199475,0.199475,0.199475,0.061923,0.061923,0.061923,1.171530,1.171530,1.171530,0.745361,0.745361,0.743463,0.738244,0.738244,0.730532,0.1
3,"[l1, l2]","[1, 2, 3, 4, 5, 6, 7]","[1, 2, 3, 4, 5, 6, 7]",{'alpha': 1.0},0.200192,0.200192,0.200192,0.058144,0.058144,0.058144,1.239896,1.239896,1.239896,0.746616,0.746616,0.746137,0.715926,0.715926,0.707888,1.0
4,"[l1, l2]","[1, 2, 3]","[1, 2, 3]",{'alpha': 10.0},0.206143,0.206143,0.206143,0.060303,0.060303,0.060303,1.267311,1.267311,1.267311,0.790667,0.790667,0.787598,0.747907,0.747907,0.750675,10.0
5,"[l1, l2]","[1, 2, 3, 4, 5, 6, 7]","[1, 2, 3, 4, 5, 6, 7]",{'alpha': 10.0},0.207645,0.207645,0.207645,0.062203,0.062203,0.062203,1.294716,1.294716,1.294716,0.773001,0.773001,0.773914,0.738505,0.738505,0.732181,10.0


...

,levels,lags,lags_label,params,mean_absolute_error__average,mean_absolute_error__weighted_average,mean_absolute_error__pooling,mean_squared_error__average,mean_squared_error__weighted_average,mean_squared_error__pooling,mean_absolute_percentage_error__average,mean_absolute_percentage_error__weighted_average,mean_absolute_percentage_error__pooling,mean_absolute_scaled_error__average,mean_absolute_scaled_error__weighted_average,mean_absolute_scaled_error__pooling,root_mean_squared_scaled_error__average,root_mean_squared_scaled_error__weighted_average,root_mean_squared_scaled_error__pooling,alpha
0,"[l1, l2]","[1, 2, 3]","[1, 2, 3]",{'alpha': 0.1},0.182203,0.182203,0.182203,0.048846,0.048846,0.048846,1.027423,1.027423,1.027423,0.697561,0.697561,0.696131,0.674418,0.674418,0.675612,0.1
1,"[l1, l2]","[1, 2, 3]","[1, 2, 3]",{'alpha': 1.0},0.182259,0.182259,0.182259,0.048839,0.048839,0.048839,1.034999,1.034999,1.034999,0.697837,0.697837,0.696346,0.674234,0.674234,0.675564,1.0
2,"[l1, l2]","[1, 2, 3]","[1, 2, 3]",{'alpha': 10.0},0.185275,0.185275,0.185275,0.050052,0.050052,0.050052,1.098270,1.098270,1.098270,0.709829,0.709829,0.707868,0.681587,0.681587,0.683902,10.0
3,"[l1, l2]","[1, 2, 3, 4, 5, 6, 7]","[1, 2, 3, 4, 5, 6, 7]",{'alpha': 10.0},0.193272,0.193272,0.193272,0.055314,0.055314,0.055314,1.207098,1.207098,1.207098,0.720035,0.720035,0.720344,0.694680,0.694680,0.690446,10.0
4,"[l1, l2]","[1, 2, 3, 4, 5, 6, 7]","[1, 2, 3, 4, 5, 6, 7]",{'alpha': 1.0},0.198754,0.198754,0.198754,0.061508,0.061508,0.061508,1.165742,1.165742,1.165742,0.741826,0.741826,0.740777,0.734049,0.734049,0.728079,1.0
5,"[l1, l2]","[1, 2, 3, 4, 5, 6, 7]","[1, 2, 3, 4, 5, 6, 7]",{'alpha': 0.1},0.203171,0.203171,0.203171,0.064074,0.064074,0.064074,1.164854,1.164854,1.164854,0.758583,0.758583,0.757238,0.749397,0.749397,0.743113,0.1


3


,levels,lags,lags_label,params,mean_absolute_error,mean_squared_error,mean_absolute_percentage_error,mean_absolute_scaled_error,root_mean_squared_scaled_error,alpha
0,[l1],"[1, 2, 3]","[1, 2, 3]",{'alpha': 0.1},0.073671,0.009153,0.180190,0.273346,0.301710,0.1
1,[l1],"[1, 2, 3]","[1, 2, 3]",{'alpha': 1.0},0.077623,0.009842,0.208891,0.288008,0.312861,1.0
2,[l1],"[1, 2, 3]","[1, 2, 3]",{'alpha': 10.0},0.110021,0.018882,0.343295,0.408219,0.433336,10.0
3,[l1],"[1, 2, 3, 4, 5, 6, 7]","[1, 2, 3, 4, 5, 6, 7]",{'alpha': 10.0},0.179616,0.046386,0.502170,0.719871,0.713240,10.0
4,[l1],"[1, 2, 3, 4, 5, 6, 7]","[1, 2, 3, 4, 5, 6, 7]",{'alpha': 1.0},0.203850,0.057269,0.550379,0.816998,0.792510,1.0
5,[l1],"[1, 2, 3, 4, 5, 6, 7]","[1, 2, 3, 4, 5, 6, 7]",{'alpha': 0.1},0.215648,0.063630,0.576964,0.864284,0.835361,0.1


/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1802: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


,levels,lags,lags_label,params,mean_absolute_error,mean_squared_error,mean_absolute_percentage_error,mean_absolute_scaled_error,root_mean_squared_scaled_error,alpha
0,[l1],"[1, 2, 3]","[1, 2, 3]",{'alpha': 0.1},0.073671,0.009153,0.180190,0.273346,0.301710,0.1
1,[l1],"[1, 2, 3]","[1, 2, 3]",{'alpha': 1.0},0.077623,0.009842,0.208891,0.288008,0.312861,1.0
2,[l1],"[1, 2, 3]","[1, 2, 3]",{'alpha': 10.0},0.110021,0.018882,0.343295,0.408219,0.433336,10.0
3,[l1],"[1, 2, 3, 4, 5, 6, 7]","[1, 2, 3, 4, 5, 6, 7]",{'alpha': 10.0},0.179616,0.046386,0.502170,0.719871,0.713240,10.0
4,[l1],"[1, 2, 3, 4, 5, 6, 7]","[1, 2, 3, 4, 5, 6, 7]",{'alpha': 1.0},0.203850,0.057269,0.550379,0.816998,0.792510,1.0
5,[l1],"[1, 2, 3, 4, 5, 6, 7]","[1, 2, 3, 4, 5, 6, 7]",{'alpha': 0.1},0.215648,0.063630,0.576964,0.864284,0.835361,0.1


In [8]:
import joblib
from pathlib import Path
# THIS_DIR = Path(__file__).parent
# series_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series.joblib')
# exog_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series_exog.joblib')
series_dict = joblib.load("/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/tests/fixture_sample_multi_series.joblib")
exog_dict = joblib.load("/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/tests/fixture_sample_multi_series_exog.joblib")
end_train = "2016-07-31 23:59:00"
initial_train_size = 213

def test_evaluate_grid_hyperparameters_equivalent_outputs_backtesting_one_step_ahead(
    forecaster,
):

    metrics = [
        "mean_absolute_error",
        "mean_squared_error",
        mean_absolute_percentage_error,
        mean_absolute_scaled_error,
        root_mean_squared_scaled_error,
    ]
    steps = 1
    initial_train_size = 213
    param_grid = {
        "n_estimators": [5, 10],
        "max_depth": [2, 3]
    }
    lags_grid = [3, 5]
    param_grid = list(ParameterGrid(param_grid))
    results_backtesting = _evaluate_grid_hyperparameters_multiseries(
        forecaster         = forecaster,
        series             = series_dict,
        exog               = exog_dict,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        steps              = steps,
        refit              = False,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'backtesting',
        aggregate_metric   = ["average", "weighted_average", "pooling"],
        fixed_train_size   = False,
        return_best        = False,
        n_jobs             = 'auto',
        verbose            = False,
        show_progress      = False
    )
    # display(results_backtesting)
    results_one_step_ahead = _evaluate_grid_hyperparameters_multiseries(
        forecaster         = forecaster,
        series             = series_dict,
        exog               = exog_dict,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'one_step_ahead',
        aggregate_metric   = ["average", "weighted_average", "pooling"],
        return_best        = False,
        verbose            = False,
        show_progress      = False
    )
    # display(results_one_step_ahead)

    pd.testing.assert_frame_equal(results_backtesting, results_one_step_ahead)


regressor = LGBMRegressor(random_state=678, verbose=-1)
forecasters = [
    ForecasterAutoregMultiSeries(regressor=LGBMRegressor(random_state=678, verbose=-1), lags=3, forecaster_id=1),
    ForecasterAutoregMultiSeries(
        regressor=LGBMRegressor(random_state=678, verbose=-1),
        lags=3,
        transformer_series=None,
        forecaster_id=2
    ),
    ForecasterAutoregMultiSeries(
        regressor=LGBMRegressor(random_state=678, verbose=-1),
        lags=3,
        transformer_series=StandardScaler(),
        transformer_exog=StandardScaler(),
        forecaster_id=3
    )
]
for forecaster in forecasters:
    print(forecaster.forecaster_id)
    test_evaluate_grid_hyperparameters_equivalent_outputs_backtesting_one_step_ahead(
        forecaster=forecaster
    )


1


/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1802: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


2


/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1802: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


3


/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:1802: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


In [9]:
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries

forecaster = ForecasterAutoregMultiSeries(
    regressor=LGBMRegressor(random_state=678, verbose=-1, **{'max_depth': 2, 'n_estimators': 10}),
    lags=1
)

metrics, predictions = backtesting_forecaster_multiseries(
    forecaster=forecaster,
    series=series_dict,
    exog=exog_dict,
    steps=1,
    initial_train_size=213,
    metric=["mean_absolute_error", mean_absolute_percentage_error, mean_absolute_scaled_error],
    add_aggregated_metric=True,
    n_jobs='auto',
    verbose=False,
    show_progress=True
)

display(metrics)

forecaster = ForecasterAutoregMultiSeries(
    regressor=LGBMRegressor(random_state=678, verbose=-1, **{'max_depth': 2, 'n_estimators': 10}),
    lags=1
)
(
    X_train,
    y_train,
    X_test,
    y_test,
    X_train_encoding,
    X_test_encoding
) = forecaster._train_test_split_one_step_ahead(
        series             = series_dict,
        exog               = exog_dict,
        initial_train_size = 213,
    )

metrics_one_step_ahead, predictions_one_step_ahead = _predict_and_calculate_metrics_multiseries_one_step_ahead(
    forecaster=forecaster,
    series=series_dict,
    X_train = X_train,
    y_train= y_train,
    X_train_encoding = X_train_encoding,
    X_test = X_test,
    y_test = y_test,
    X_test_encoding = X_test_encoding,
    levels = ['id_1000', 'id_1001', 'id_1002', 'id_1003', 'id_1004'],
    metrics=["mean_absolute_error", mean_absolute_percentage_error, mean_absolute_scaled_error],
    add_aggregated_metric = True
)
display(metrics_one_step_ahead)


pd.testing.assert_frame_equal(metrics, metrics_one_step_ahead)

  0%|          | 0/153 [00:00<?, ?it/s]

,levels,mean_absolute_error,mean_absolute_percentage_error,mean_absolute_scaled_error
0,id_1000,653.625859,5.121216e-01,2.963779
1,id_1001,1769.398384,7.115652e+16,4.857173
2,id_1002,NaN,NaN,NaN
3,id_1003,628.248235,3.441142e-01,2.656275
4,id_1004,1746.409857,2.157294e-01,1.447493
5,average,1199.420584,1.778913e+16,2.981180
6,weighted_average,1139.379553,2.610779e+16,3.486763
7,pooling,1139.379553,2.610779e+16,2.696035


,levels,mean_absolute_error,mean_absolute_percentage_error,mean_absolute_scaled_error
0,id_1000,653.625859,5.121216e-01,2.963779
1,id_1001,1769.398384,7.115652e+16,4.857173
2,id_1002,NaN,NaN,NaN
3,id_1003,628.248235,3.441142e-01,2.656275
4,id_1004,1746.409857,2.157294e-01,1.447493
5,average,1199.420584,1.778913e+16,2.981180
6,weighted_average,1139.379553,2.610779e+16,3.486763
7,pooling,1139.379553,2.610779e+16,2.696035


## backtesting vs _predict_and_calculate_metrics_multiseries_one_step_ahead

In [6]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from skforecast.model_selection_multiseries import backtesting_forecaster_multiseries
from skforecast.model_selection_multiseries.model_selection_multiseries import _predict_and_calculate_metrics_multiseries_one_step_ahead
from skforecast.model_selection_multiseries.model_selection_multiseries import _calculate_metrics_multiseries
from skforecast.model_selection.model_selection import _create_backtesting_folds
from skforecast.metrics import add_y_train_argument
from skforecast.metrics import mean_absolute_scaled_error

In [14]:
series = fetch_dataset(name="items_sales")
series.to_parquet("fixture_items_sales.parquet")
series.index

items_sales
-----------
Simulated time series for the sales of 3 different items.
Simulated data.
Shape of the dataset: (1097, 3)


DatetimeIndex(['2012-01-01', '2012-01-02', '2012-01-03', '2012-01-04',
               '2012-01-05', '2012-01-06', '2012-01-07', '2012-01-08',
               '2012-01-09', '2012-01-10',
               ...
               '2014-12-23', '2014-12-24', '2014-12-25', '2014-12-26',
               '2014-12-27', '2014-12-28', '2014-12-29', '2014-12-30',
               '2014-12-31', '2015-01-01'],
              dtype='datetime64[ns]', name='date', length=1097, freq='D')

In [16]:
series = pd.read_parquet("fixture_items_sales.parquet")
series.asfreq('D')

,item_1,item_2,item_3
date,,,
2012-01-01,8.253175,21.047727,19.429739
2012-01-02,22.777826,26.578125,28.009863
2012-01-03,27.549099,31.751042,32.078922
2012-01-04,25.895533,24.567708,27.252276
2012-01-05,21.379238,18.191667,20.357737
...,...,...,...
2014-12-28,17.329233,18.189583,20.586030
2014-12-29,19.611623,24.539583,28.127390
2014-12-30,18.857026,17.677083,21.555782


In [18]:
# Test results _predict_and_calculate_metrics_multiseries_one_step_ahead and backtesting_forecaster_multiseries
# ==============================================================================
# Metrics and predictions should be equal when using step=1 and refit=False
# in backtesting_forecaster_multiseries and _predict_and_calculate_metrics_multiseries_one_step_ahead


# Data download
series = pd.read_parquet("fixture_items_sales.parquet")
series = series.asfreq('D')
exog = pd.DataFrame(
    {
        'day_of_week': series.index.dayofweek
    },
    index = series.index
)
initial_train_size = 927

# Metrics with _calculate_metrics_multiseries_one_step_ahead
metrics = [mean_absolute_error, mean_absolute_percentage_error, mean_absolute_scaled_error]

forecasters = [
            ForecasterAutoregMultiSeries(regressor=Ridge(random_state=678), lags=3),
            ForecasterAutoregMultiSeries(
                regressor=Ridge(random_state=678),
                lags=3,
                transformer_series=None,
            ),
            ForecasterAutoregMultiSeries(
                regressor=Ridge(random_state=678),
                lags=3,
                transformer_series=StandardScaler(),
                transformer_exog=StandardScaler()
            ),
            ForecasterAutoregMultiVariate(
                regressor=Ridge(random_state=678),
                level='item_1',
                lags=3,
                steps=1,
                transformer_series=StandardScaler(),
                transformer_exog=StandardScaler()
            )
        ]

for forecaster in forecasters:

    if type(forecaster) is ForecasterAutoregMultiSeries:
        levels = ['item_1', 'item_2', 'item_3']
    else:
        levels = ['item_1']

    metrics_backtesting, predictions = backtesting_forecaster_multiseries(
        series=series,
        exog=exog,
        forecaster=forecaster,
        steps=1,
        metric=metrics,
        initial_train_size = initial_train_size,
        refit=False,
        levels=levels,
        add_aggregated_metric=True,
        show_progress=False
    )

    display(metrics_backtesting)

    (
        X_train,
        y_train,
        X_test,
        y_test,
        X_train_encoding,
        X_test_encoding
    ) = forecaster._train_test_split_one_step_ahead(
            series             = series,
            exog               = exog,
            initial_train_size = initial_train_size,
        )

    metrics_one_step_ahead, predictions_one_step_ahead = _predict_and_calculate_metrics_multiseries_one_step_ahead(
        forecaster=forecaster,
        series=series,
        X_train = X_train,
        y_train= y_train,
        X_train_encoding = X_train_encoding,
        X_test = X_test,
        y_test = y_test,
        X_test_encoding = X_test_encoding,
        levels = levels,
        metrics = metrics,
        add_aggregated_metric = True
    )

    display(metrics_one_step_ahead)


    pd.testing.assert_frame_equal(metrics_one_step_ahead, metrics_backtesting)
    pd.testing.assert_frame_equal(predictions_one_step_ahead, predictions)

/home/ubuntu/varios/skforecast/skforecast/ForecasterAutoregMultiSeries/ForecasterAutoregMultiSeries.py:400: UserWarning: When using a linear model, it is recommended to use a transformer_series to ensure all series are in the same scale. You can use, for example, a `StandardScaler` from sklearn.preprocessing.
  warnings.warn(


,levels,mean_absolute_error,mean_absolute_percentage_error,mean_absolute_scaled_error
0,item_1,1.192201,0.057958,0.778281
1,item_2,2.369540,0.161063,1.004057
2,item_3,3.316136,0.220772,0.892257
3,average,2.292626,0.146597,0.891532
4,weighted_average,2.292626,0.146597,0.891532
5,pooling,2.292626,0.146597,0.903988


,levels,mean_absolute_error,mean_absolute_percentage_error,mean_absolute_scaled_error
0,item_1,1.192201,0.057958,0.778281
1,item_2,2.369540,0.161063,1.004057
2,item_3,3.316136,0.220772,0.892257
3,average,2.292626,0.146597,0.891532
4,weighted_average,2.292626,0.146597,0.891532
5,pooling,2.292626,0.146597,0.903988


,levels,mean_absolute_error,mean_absolute_percentage_error,mean_absolute_scaled_error
0,item_1,1.192201,0.057958,0.778281
1,item_2,2.369540,0.161063,1.004057
2,item_3,3.316136,0.220772,0.892257
3,average,2.292626,0.146597,0.891532
4,weighted_average,2.292626,0.146597,0.891532
5,pooling,2.292626,0.146597,0.903988


,levels,mean_absolute_error,mean_absolute_percentage_error,mean_absolute_scaled_error
0,item_1,1.192201,0.057958,0.778281
1,item_2,2.369540,0.161063,1.004057
2,item_3,3.316136,0.220772,0.892257
3,average,2.292626,0.146597,0.891532
4,weighted_average,2.292626,0.146597,0.891532
5,pooling,2.292626,0.146597,0.903988


,levels,mean_absolute_error,mean_absolute_percentage_error,mean_absolute_scaled_error
0,item_1,1.117062,0.055294,0.729230
1,item_2,2.308444,0.153721,0.978168
2,item_3,3.491733,0.234365,0.939504
3,average,2.305746,0.147793,0.882301
4,weighted_average,2.305746,0.147793,0.882301
5,pooling,2.305746,0.147793,0.909161


,levels,mean_absolute_error,mean_absolute_percentage_error,mean_absolute_scaled_error
0,item_1,1.117062,0.055294,0.729230
1,item_2,2.308444,0.153721,0.978168
2,item_3,3.491733,0.234365,0.939504
3,average,2.305746,0.147793,0.882301
4,weighted_average,2.305746,0.147793,0.882301
5,pooling,2.305746,0.147793,0.909161


,levels,mean_absolute_error,mean_absolute_percentage_error,mean_absolute_scaled_error
0,item_1,0.889368,0.043968,0.580589


,levels,mean_absolute_error,mean_absolute_percentage_error,mean_absolute_scaled_error
0,item_1,0.889368,0.043968,0.580589


In [9]:
def test_predict_and_calculate_metrics_multiseries_one_step_ahead_output_equivalence_to_backtesting(forecaster):
    """
    Test that the output of _predict_and_calculate_metrics_multiseries_one_step_ahead is equivalent to
    the output of backtesting_forecaster_multiseries when steps=1 and refit=False.

    Results are not equivalent if diferentiation is included
    """

    initial_train_size = 927
    metrics = ['mean_absolute_error', mean_absolute_percentage_error, mean_absolute_scaled_error]

    if type(forecaster) is ForecasterAutoregMultiSeries:
        levels = ['item_1', 'item_2', 'item_3']
    else:
        levels = ['item_1']

    metrics_backtesting, pred_backtesting = backtesting_forecaster_multiseries(
        series=series,
        exog=exog,
        forecaster=forecaster,
        steps=1,
        metric=metrics,
        initial_train_size = initial_train_size,
        refit=False,
        levels=levels,
        add_aggregated_metric=True,
        show_progress=False
    )

    (
        X_train,
        y_train,
        X_test,
        y_test,
        X_train_encoding,
        X_test_encoding
    ) = forecaster._train_test_split_one_step_ahead(
            series             = series,
            exog               = exog,
            initial_train_size = initial_train_size,
        )

    metrics_one_step_ahead, pred_one_step_ahead = _predict_and_calculate_metrics_multiseries_one_step_ahead(
        forecaster=forecaster,
        series=series,
        X_train = X_train,
        y_train= y_train,
        X_train_encoding = X_train_encoding,
        X_test = X_test,
        y_test = y_test,
        X_test_encoding = X_test_encoding,
        levels = levels,
        metrics = metrics,
        add_aggregated_metric = True
    )


    pd.testing.assert_frame_equal(metrics_one_step_ahead, metrics_backtesting)
    pd.testing.assert_frame_equal(pred_one_step_ahead, pred_backtesting)

for forecaster in forecasters:
    test_predict_and_calculate_metrics_multiseries_one_step_ahead_output_equivalence_to_backtesting(forecaster)

In [13]:
predictions_one_step_ahead.index

DatetimeIndex(['2014-07-16', '2014-07-17', '2014-07-18', '2014-07-19',
               '2014-07-20', '2014-07-21', '2014-07-22', '2014-07-23',
               '2014-07-24', '2014-07-25',
               ...
               '2014-12-23', '2014-12-24', '2014-12-25', '2014-12-26',
               '2014-12-27', '2014-12-28', '2014-12-29', '2014-12-30',
               '2014-12-31', '2015-01-01'],
              dtype='datetime64[ns]', length=170, freq='D')

In [14]:
predictions.index

DatetimeIndex(['2014-07-16', '2014-07-17', '2014-07-18', '2014-07-19',
               '2014-07-20', '2014-07-21', '2014-07-22', '2014-07-23',
               '2014-07-24', '2014-07-25',
               ...
               '2014-12-23', '2014-12-24', '2014-12-25', '2014-12-26',
               '2014-12-27', '2014-12-28', '2014-12-29', '2014-12-30',
               '2014-12-31', '2015-01-01'],
              dtype='datetime64[ns]', length=170, freq='D')

In [15]:
# Test results _predict_and_calculate_metrics_multiseries_one_step_ahead and backtesting_forecaster_multiseries
# ==============================================================================
# Metrics and predictions should be equal when using step=1 and refit=False in backtesting_forecaster_multiseries


# Data download
import joblib
from pathlib import Path
# THIS_DIR = Path(__file__).parent
# series_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series.joblib')
# exog_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series_exog.joblib')
series_dict = joblib.load("/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/tests/fixture_sample_multi_series.joblib")
exog_dict = joblib.load("/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/tests/fixture_sample_multi_series_exog.joblib")
end_train = "2016-07-31 23:59:00"
initial_train_size = 213

# Metrics with _calculate_metrics_multiseries_one_step_ahead
metrics = [mean_absolute_error, mean_absolute_percentage_error, mean_absolute_scaled_error]
metrics = [add_y_train_argument(metric) for metric in metrics]

forecasters = [
    ForecasterAutoregMultiSeries(
        regressor          = LGBMRegressor(random_state=123, verbose=-1),
        lags               = 24,
        encoding           = 'ordinal',
        transformer_series = StandardScaler(),
        transformer_exog   = StandardScaler(),
        weight_func        = None,
        series_weights     = None,
        differentiation    = None,
        dropna_from_series = False,
    ),

]

for forecaster in forecasters:

    if type(forecaster) is ForecasterAutoregMultiSeries:
        levels = ['id_1000', 'id_1001', 'id_1002', 'id_1003', 'id_1004']
    else:
        levels = ['id_1000']

    metrics_backtesting, predictions = backtesting_forecaster_multiseries(
        series=series_dict,
        exog=exog_dict,
        forecaster=forecaster,
        steps=1,
        metric=metrics,
        initial_train_size = initial_train_size,
        refit=False,
        levels=levels,
        add_aggregated_metric=True,
        show_progress=False
    )

    display(metrics_backtesting)

    (
        X_train,
        y_train,
        X_test,
        y_test,
        X_train_encoding,
        X_test_encoding
    ) = forecaster._train_test_split_one_step_ahead(
            series             = series_dict,
            exog               = exog_dict,
            initial_train_size = initial_train_size,
        )

    metrics_one_step_ahead, predictions_one_step_ahead = _predict_and_calculate_metrics_multiseries_one_step_ahead(
        forecaster=forecaster,
        series=series_dict,
        X_train = X_train,
        y_train= y_train,
        X_train_encoding = X_train_encoding,
        X_test = X_test,
        y_test = y_test,
        X_test_encoding = X_test_encoding,
        levels = levels,
        metrics = metrics,
        add_aggregated_metric = True
    )

    display(metrics_one_step_ahead)


    pd.testing.assert_frame_equal(metrics_one_step_ahead, metrics_backtesting)
    pd.testing.assert_frame_equal(predictions_one_step_ahead, predictions)

,levels,mean_absolute_error,mean_absolute_percentage_error,mean_absolute_scaled_error
0,id_1000,128.956250,8.742856e-02,0.615734
1,id_1001,901.076173,3.344592e+16,5.935943
2,id_1002,NaN,NaN,NaN
3,id_1003,196.715372,1.004418e-01,0.788344
4,id_1004,804.919558,1.220198e-01,0.635045
5,average,507.916838,8.361481e+15,1.993766
6,weighted_average,475.502870,1.227153e+16,2.602303
7,pooling,475.502870,1.227153e+16,1.165064


,levels,mean_absolute_error,mean_absolute_percentage_error,mean_absolute_scaled_error
0,id_1000,128.956250,8.742856e-02,0.615734
1,id_1001,901.076173,3.344592e+16,5.935943
2,id_1002,NaN,NaN,NaN
3,id_1003,196.715372,1.004418e-01,0.788344
4,id_1004,804.919558,1.220198e-01,0.635045
5,average,507.916838,8.361481e+15,1.993766
6,weighted_average,475.502870,1.227153e+16,2.602303
7,pooling,475.502870,1.227153e+16,1.165064


In [16]:
# Test results _predict_and_calculate_metrics_multiseries_one_step_ahead and backtesting_forecaster_multiseries
# ==============================================================================
# Metrics and predictions should be equal when using step=1 and refit=False in backtesting_forecaster_multiseries


# Data download
import joblib
from pathlib import Path
# THIS_DIR = Path(__file__).parent
# series_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series.joblib')
# exog_dict = joblib.load(THIS_DIR/'fixture_sample_multi_series_exog.joblib')
series_dict = joblib.load("/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/tests/fixture_sample_multi_series.joblib")
exog_dict = joblib.load("/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/tests/fixture_sample_multi_series_exog.joblib")
end_train = "2016-07-31 23:59:00"
initial_train_size = 213

# Metrics with _calculate_metrics_multiseries_one_step_ahead
metrics = [mean_absolute_error, mean_absolute_percentage_error, mean_absolute_scaled_error]
metrics = [add_y_train_argument(metric) for metric in metrics]

forecaster = ForecasterAutoregMultiSeries(
        regressor          = Ridge(random_state=123),
        lags               = 10,
        encoding           = 'ordinal',
        transformer_series = StandardScaler(),
        transformer_exog   = StandardScaler(),
        weight_func        = None,
        series_weights     = None,
        differentiation    = None,
        dropna_from_series = True,
    )


metrics_backtesting, predictions = backtesting_forecaster_multiseries(
    series=series_dict,
    #exog=exog_dict,
    forecaster=forecaster,
    steps=10,
    metric=metrics,
    initial_train_size = initial_train_size,
    refit=False,
    levels=['id_1000', 'id_1001', 'id_1002', 'id_1003', 'id_1004'],
    add_aggregated_metric=False,
    show_progress=False
)

display(metrics_backtesting)

,levels,mean_absolute_error,mean_absolute_percentage_error,mean_absolute_scaled_error
0,id_1000,187.154572,1.317794e-01,0.868741
1,id_1001,943.027968,3.296872e+16,2.761642
2,id_1002,NaN,NaN,NaN
3,id_1003,203.228478,1.087644e-01,0.848920
4,id_1004,839.062142,1.352426e-01,0.680035


In [17]:
initial_train_size = 213
forecaster = ForecasterAutoregMultiSeries(
        regressor          = LGBMRegressor(random_state=123, verbose=-1),
        lags               = 10,
        encoding           = 'ordinal',
        transformer_series = StandardScaler(),
        transformer_exog   = StandardScaler(),
        weight_func        = None,
        series_weights     = None,
        differentiation    = None,
        dropna_from_series = True,
    )

(
    X_train,
    y_train,
    X_test,
    y_test,
    X_train_encoding,
    X_test_encoding
) = forecaster._train_test_split_one_step_ahead(
        series             = series_dict,
        #exog               = exog_dict,
        initial_train_size = initial_train_size,
    )

metrics_one_step_ahead, predictions_one_step_ahead = _predict_and_calculate_metrics_multiseries_one_step_ahead(
    forecaster=forecaster,
    series=series_dict,
    X_train = X_train,
    y_train= y_train,
    X_train_encoding = X_train_encoding,
    X_test = X_test,
    y_test = y_test,
    X_test_encoding = X_test_encoding,
    levels = levels,
    metrics = metrics,
    add_aggregated_metric = True
)

metrics_one_step_ahead

,levels,mean_absolute_error,mean_absolute_percentage_error,mean_absolute_scaled_error
0,id_1000,138.459914,9.302698e-02,0.642708
1,id_1001,1326.348427,5.852032e+16,3.884190
2,id_1002,NaN,NaN,NaN
3,id_1003,194.510963,1.086343e-01,0.786311
4,id_1004,684.720990,1.037579e-01,0.554946
5,average,586.010073,1.463008e+16,1.467039
6,weighted_average,636.259937,2.199904e+16,1.879264
7,pooling,636.259937,2.199904e+16,1.498528


In [18]:
forecaster.encoder.categories_

[array(['id_1000', 'id_1001', 'id_1002', 'id_1003', 'id_1004'],
       dtype=object)]

In [19]:
from skforecast.model_selection_multiseries import bayesian_search_forecaster_multiseries

series_item_sales = fetch_dataset(name="items_sales", verbose=False)
exog_item_sales = pd.DataFrame({'day_of_week': series_item_sales.index.dayofweek}, index = series_item_sales.index)

levels = ["item_1", "item_2", "item_3"]
exog_features = ['day_of_week']


metrics = ['mean_absolute_error', mean_absolute_percentage_error, mean_absolute_scaled_error]
forecaster = ForecasterAutoregMultiSeries(
            regressor          = Ridge(random_state=123),
            lags               = 3,
            encoding           = 'ordinal',
            transformer_series = StandardScaler(),
            transformer_exog   = StandardScaler(),
            weight_func        = None,
            series_weights     = None,
            differentiation    = None,
            dropna_from_series = False,
        )

initial_train_size = 1000
levels = ["item_1", "item_2", "item_3"]

def search_space(trial):
    search_space  = {
        'alpha': trial.suggest_float('alpha', 1e-3, 1e3, log=True),
        'lags': trial.suggest_categorical('lags', [3, 5]),
    }
    
    return search_space


results, best_trial = bayesian_search_forecaster_multiseries(
    forecaster         = forecaster,
    series             = series_item_sales,
    exog               = exog_item_sales,
    search_space       = search_space,
    n_trials           = 5,
    metric             = metrics,
    initial_train_size = initial_train_size,
    levels             = levels,
    method             = 'one_step_ahead',
    aggregate_metric   = ["average", "weighted_average", "pooling"],
    return_best        = False,
    n_jobs             = 'auto',
    verbose            = False,
    show_progress      = False
)

print(results.index)
results.to_dict(orient='list')

/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:2349: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


RangeIndex(start=0, stop=5, step=1)


{'levels': [['item_1', 'item_2', 'item_3'],
  ['item_1', 'item_2', 'item_3'],
  ['item_1', 'item_2', 'item_3'],
  ['item_1', 'item_2', 'item_3'],
  ['item_1', 'item_2', 'item_3']],
 'lags': [array([1, 2, 3]),
  array([1, 2, 3, 4, 5]),
  array([1, 2, 3, 4, 5]),
  array([1, 2, 3]),
  array([1, 2, 3])],
 'params': [{'alpha': 766.6289057556013},
  {'alpha': 0.4279898484823994},
  {'alpha': 0.2252709077935534},
  {'alpha': 15.094374246471325},
  {'alpha': 2.031835829826598}],
 'mean_absolute_error__average': [2.487682902584588,
  2.5778104346448623,
  2.5779063732830942,
  2.599406545255343,
  2.605166414875337],
 'mean_absolute_error__weighted_average': [2.487682902584588,
  2.5778104346448623,
  2.5779063732830947,
  2.599406545255343,
  2.6051664148753373],
 'mean_absolute_error__pooling': [2.4876829025845875,
  2.5778104346448623,
  2.5779063732830947,
  2.599406545255343,
  2.6051664148753373],
 'mean_absolute_percentage_error__average': [0.13537962851907534,
  0.14171211559040905,
  0

In [20]:
def test_output_bayesian_search_forecaster_multiseries_ForecasterAutoregMultiSeries_one_step_ahead():
    """
    Test output of bayesian_search_forecaster_multiseries when forecaster is ForecasterAutoregMultiSeries
    and method is one_step_ahead.
    """
    metrics = ['mean_absolute_error', mean_absolute_percentage_error, mean_absolute_scaled_error]
    forecaster = ForecasterAutoregMultiSeries(
            regressor          = Ridge(random_state=123),
            lags               = 3,
            encoding           = 'ordinal',
            transformer_series = StandardScaler(),
            transformer_exog   = StandardScaler(),
            weight_func        = None,
            series_weights     = None,
            differentiation    = None,
            dropna_from_series = False,
        )

    initial_train_size = 1000
    levels = ["item_1", "item_2", "item_3"]

    def search_space(trial):
        search_space  = {
            'alpha': trial.suggest_float('alpha', 1e-3, 1e3, log=True),
            'lags': trial.suggest_categorical('lags', [3, 5]),
        }

        return search_space

    results, _ = bayesian_search_forecaster_multiseries(
        forecaster         = forecaster,
        series             = series_item_sales,
        exog               = exog_item_sales,
        search_space       = search_space,
        n_trials           = 5,
        metric             = metrics,
        levels             = levels,
        initial_train_size = initial_train_size,
        method             = 'one_step_ahead',
        aggregate_metric   = ["average", "weighted_average", "pooling"],
        return_best        = False,
        n_jobs             = 'auto',
        verbose            = False,
        show_progress      = False
    )

    expected_results = pd.DataFrame(
        {
            "levels": [
                ["item_1", "item_2", "item_3"],
                ["item_1", "item_2", "item_3"],
                ["item_1", "item_2", "item_3"],
                ["item_1", "item_2", "item_3"],
                ["item_1", "item_2", "item_3"],
            ],
            "lags": [
                np.array([1, 2, 3]),
                np.array([1, 2, 3, 4, 5]),
                np.array([1, 2, 3, 4, 5]),
                np.array([1, 2, 3]),
                np.array([1, 2, 3]),
            ],
            "params": [
                {"alpha": 766.6289057556013},
                {"alpha": 0.4279898484823994},
                {"alpha": 0.2252709077935534},
                {"alpha": 15.094374246471325},
                {"alpha": 2.031835829826598},
            ],
            "mean_absolute_error__average": [
                2.487682902584588,
                2.5778104346448623,
                2.5779063732830942,
                2.599406545255343,
                2.605166414875337,
            ],
            "mean_absolute_error__weighted_average": [
                2.487682902584588,
                2.5778104346448623,
                2.5779063732830947,
                2.599406545255343,
                2.6051664148753373,
            ],
            "mean_absolute_error__pooling": [
                2.4876829025845875,
                2.5778104346448623,
                2.5779063732830947,
                2.599406545255343,
                2.6051664148753373,
            ],
            "mean_absolute_percentage_error__average": [
                0.13537962851907534,
                0.14171211559040905,
                0.1417181361315976,
                0.1425099620058197,
                0.14287737387127633,
            ],
            "mean_absolute_percentage_error__weighted_average": [
                0.13537962851907534,
                0.14171211559040905,
                0.14171813613159756,
                0.14250996200581972,
                0.14287737387127633,
            ],
            "mean_absolute_percentage_error__pooling": [
                0.13537962851907534,
                0.14171211559040905,
                0.14171813613159756,
                0.14250996200581975,
                0.14287737387127633,
            ],
            "mean_absolute_scaled_error__average": [
                0.9807885628114613,
                0.9887040054273549,
                0.9887272793513265,
                0.9982897158009892,
                0.999726922014544,
            ],
            "mean_absolute_scaled_error__weighted_average": [
                0.9807885628114613,
                0.9887040054273548,
                0.9887272793513265,
                0.9982897158009892,
                0.9997269220145439,
            ],
            "mean_absolute_scaled_error__pooling": [
                0.994386675351447,
                1.0309527259247835,
                1.0309910949992809,
                1.039045301850066,
                1.0413476602398484,
            ],
            "alpha": [
                766.6289057556013,
                0.4279898484823994,
                0.2252709077935534,
                15.094374246471325,
                2.031835829826598,
            ],
        },
        index=pd.RangeIndex(start=0, stop=5, step=1),
    )

    pd.testing.assert_frame_equal(results, expected_results)

test_output_bayesian_search_forecaster_multiseries_ForecasterAutoregMultiSeries_one_step_ahead()

/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/model_selection_multiseries.py:2349: UserWarning: One-step-ahead predictions are used for faster model comparison, but they may not fully represent multi-step prediction performance. It is recommended to backtest the final model for a more accurate multi-step performance estimate.
  warnings.warn(


In [21]:
series_dict = joblib.load("/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/tests/fixture_sample_multi_series.joblib")
exog_dict = joblib.load("/home/ubuntu/varios/skforecast/skforecast/model_selection_multiseries/tests/fixture_sample_multi_series_exog.joblib")
end_train = "2016-07-31 23:59:00"
initial_train_size = 213

def test_output_bayesian_search_forecaster_multiseries_ForecasterAutoregMultiSeries_one_step_ahead_series_is_dict():
    """
    Test output of grid_search_forecaster_multiseries in ForecasterAutoregMultiSeriesCustom
    with mocked (mocked done in Skforecast v0.5.0)
    """
    metrics = ['mean_absolute_error', mean_absolute_percentage_error, mean_absolute_scaled_error]
    forecaster = ForecasterAutoregMultiSeries(
            regressor          = LGBMRegressor(random_state=678, verbose=-1),
            lags               = 3,
            encoding           = 'ordinal',
            transformer_series = StandardScaler(),
            transformer_exog   = StandardScaler(),
            weight_func        = None,
            series_weights     = None,
            differentiation    = None,
            dropna_from_series = False,
        )
    steps = 10
    initial_train_size = 213
    levels = ["id_1000", "id_1001", "id_1002", "id_1003", "id_1004"]
    param_grid = {
            "n_estimators": [5, 10],
            "max_depth": [2, 3]
        }
    lags_grid = [3, 5]

    results = grid_search_forecaster_multiseries(
        forecaster         = forecaster,
        series             = series_dict,
        exog               = exog_dict,
        param_grid         = param_grid,
        lags_grid          = lags_grid,
        steps              = steps,
        refit              = False,
        metric             = metrics,
        initial_train_size = initial_train_size,
        method             = 'one_step_ahead',
        aggregate_metric   = ["average", "weighted_average", "pooling"],
        levels            = levels,
        fixed_train_size   = False,
        return_best        = False,
        n_jobs             = 'auto',
        verbose            = False,
        show_progress      = False
    )
    
    expected_results = pd.DataFrame({
        "levels": [
            ["id_1000", "id_1001", "id_1002", "id_1003", "id_1004"],
            ["id_1000", "id_1001", "id_1002", "id_1003", "id_1004"],
            ["id_1000", "id_1001", "id_1002", "id_1003", "id_1004"],
            ["id_1000", "id_1001", "id_1002", "id_1003", "id_1004"],
            ["id_1000", "id_1001", "id_1002", "id_1003", "id_1004"],
            ["id_1000", "id_1001", "id_1002", "id_1003", "id_1004"],
            ["id_1000", "id_1001", "id_1002", "id_1003", "id_1004"],
            ["id_1000", "id_1001", "id_1002", "id_1003", "id_1004"],
        ],
        "lags": [
            np.array([1, 2, 3, 4, 5]),
            np.array([1, 2, 3]),
            np.array([1, 2, 3]),
            np.array([1, 2, 3, 4, 5]),
            np.array([1, 2, 3, 4, 5]),
            np.array([1, 2, 3]),
            np.array([1, 2, 3]),
            np.array([1, 2, 3, 4, 5]),
        ],
        "lags_label": [
            np.array([1, 2, 3, 4, 5]),
            np.array([1, 2, 3]),
            np.array([1, 2, 3]),
            np.array([1, 2, 3, 4, 5]),
            np.array([1, 2, 3, 4, 5]),
            np.array([1, 2, 3]),
            np.array([1, 2, 3]),
            np.array([1, 2, 3, 4, 5]),
        ],
        "params": [
            {"max_depth": 3, "n_estimators": 10},
            {"max_depth": 3, "n_estimators": 10},
            {"max_depth": 2, "n_estimators": 10},
            {"max_depth": 2, "n_estimators": 10},
            {"max_depth": 3, "n_estimators": 5},
            {"max_depth": 3, "n_estimators": 5},
            {"max_depth": 2, "n_estimators": 5},
            {"max_depth": 2, "n_estimators": 5},
        ],
        "mean_absolute_error__average": [
            595.4248731239852,
            601.61658409234,
            641.5971327961081,
            643.5830924971917,
            677.9145593017354,
            687.1976066932838,
            709.8383108516048,
            712.5538939535741,
        ],
        "mean_absolute_error__weighted_average": [
            601.0754794775827,
            602.8512611773792,
            637.9906397322952,
            635.3319731010223,
            669.4535161312964,
            674.4638410667131,
            694.4718207943295,
            695.1809840497659,
        ],
        "mean_absolute_error__pooling": [
            601.0754794775827,
            602.8512611773791,
            637.9906397322952,
            635.3319731010222,
            669.4535161312965,
            674.4638410667131,
            694.4718207943295,
            695.1809840497659,
        ],
        "mean_absolute_percentage_error__average": [
            1.13122611897686e16,
            1.1653837498770236e16,
            1.1977069509022024e16,
            1.2092182501085004e16,
            1.2255524771350234e16,
            1.2487020718197592e16,
            1.3255138678699688e16,
            1.320416830274058e16,
        ],
        "mean_absolute_percentage_error__weighted_average": [
            1.6602167501530896e16,
            1.7103473739202362e16,
            1.757785740892441e16,
            1.7746800217419718e16,
            1.7986525563708256e16,
            1.832627501087992e16,
            1.945358482341537e16,
            1.9378779379561716e16,
        ],
        "mean_absolute_percentage_error__pooling": [
            1.6602167501530896e16,
            1.7103473739202362e16,
            1.7577857408924412e16,
            1.7746800217419718e16,
            1.7986525563708258e16,
            1.832627501087992e16,
            1.9453584823415372e16,
            1.9378779379561716e16,
        ],
        "mean_absolute_scaled_error__average": [
            1.338257547877582,
            1.3458574988611296,
            1.418344242255843,
            1.4196630001824362,
            1.502472367026163,
            1.516064192858667,
            1.5547477191379595,
            1.5594122630466858,
        ],
        "mean_absolute_scaled_error__weighted_average": [
            1.6314647598931409,
            1.627988835703755,
            1.720233806106665,
            1.7212080197022548,
            1.8204941416870104,
            1.8231712390456873,
            1.8766737381944514,
            1.8860125959513818,
        ],
        "mean_absolute_scaled_error__pooling": [
            1.4086529286833727,
            1.4224992984312164,
            1.505414844206229,
            1.4889348761536338,
            1.5689005596025196,
            1.5914777035729348,
            1.6386889129680593,
            1.6291942735674578,
        ],
        "max_depth": [3, 3, 2, 2, 3, 3, 2, 2],
        "n_estimators": [10, 10, 10, 10, 5, 5, 5, 5],
        },
        index = pd.RangeIndex(start=0, stop=8, step=1)
    )

    pd.testing.assert_frame_equal(results, expected_results)

test_output_grid_search_forecaster_multiseries_ForecasterAutoregMultiSeries_one_step_ahead_series_is_dict()

NameError: name 'test_output_grid_search_forecaster_multiseries_ForecasterAutoregMultiSeries_one_step_ahead_series_is_dict' is not defined